In [1]:
from sklearn import datasets
from time import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Importing our own
from models.trainer import TrainModelWrapper
from models.classification import Classification, Classification_Dropout
from models.regression import Regression, Regression_Dropout
import utils.datasets as DB

## Classification

In [2]:
datasets = DB.Datasets()
train_mnist, test_mnist = datasets.get_MNIST()
train_mnist, val_mnist = torch.utils.data.random_split(train_mnist, [50000, 10000])

In [3]:
# Multi Class Problem
class NeuralNetwork(nn.Module):
    
    def __init__(self,input_dim,hidden_nodes, num_classes, dropout=False):
        super(NeuralNetwork,self).__init__()
        
        self.input_dim = input_dim
        self.dropout = dropout
        self.linear1 = nn.Linear(input_dim,hidden_nodes)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(hidden_nodes,hidden_nodes)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(hidden_nodes,num_classes)
        if dropout:
            self.drop1 = nn.Dropout(0.2)
            self.drop2 = nn.Dropout(0.5)
            self.drop3 = nn.Dropout(0.5)
    
    def forward(self,x):
        x = x.view(-1, self.input_dim)
        if self.dropout:
            x = self.drop1(x)
        out = self.linear1(x)
        
        if self.dropout:
            out = self.drop2(out)
        out = self.relu1(out)
        out = self.linear2(out)
        
        if self.dropout:
            out = self.drop3(out)
        out = self.relu2(out)
        out = self.linear3(out)
        # NO SOFTMAX since done by crossentropyloss
        return out

In [4]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity="relu")
        m.bias.data.fill_(0)


def train_save_classification(nodes_hidden, learning_rate, name, dropout=False):
    num_epochs = 600
    batch_size = 128
    
    if not dropout:
        model = Classification(input_dim=784, output_dim=10, hl_units=nodes_hidden, hl_type=nn.Linear)
    else:
        model = Classification_Dropout(input_dim=784, output_dim=10, hl_units=nodes_hidden, hl_type=nn.Linear)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate,nesterov=True,momentum=0.9)
    model.apply(init_weights)
    args = {
            'model': model,
            'model_name':name,
            'train_dataset': train_mnist,
            'criterion': criterion,
            'batch_size': batch_size,
            'optimizer': optimizer,
            'es_flag': False,
            'num_epochs': num_epochs,
            'val_dataset': val_mnist,
            'test_dataset': test_mnist,
            'mode': 2
            }
    trainer = TrainModelWrapper(**args)
    model, history = trainer.train()
    
    torch.save(model.state_dict(), 'weights/'+name+'.pth')
    return model, history, trainer
    

In [8]:
lrs = [1e-3]
layers = [800]

for lr in lrs:
    for hidden_size in layers:
        _name = f"{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_classification(nodes_hidden=hidden_size, learning_rate=lr, name=_name)
        

2023-02-28 02:37:46,752 [WARNING] Scheduler Object not found amongst the Arguments. Ignore warning if scheduler wasn't meant to be in the loop


800_1e-03
The tensorboard model name corresponding to this model is 800_1e-03_20230228023746752069
Epoch 1/600
----------


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:05<00:00, 66.14it/s]


train Loss: 0.9777
train Acc: 0.7604


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 71.36it/s]


...

train Loss: 0.0009
train Acc: 1.0000


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 66.72it/s]


val Loss: 0.0863
val Acc: 0.9802


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 65.78it/s]

test Loss: 0.0751
test Acc: 0.9818

Training completed in 1h 28m 22s


In [5]:
lrs = [1e-3]
layers = [800]
for lr in lrs:
    for hidden_size in layers:
        _name = f"drop_{hidden_size}_{lr:.0e}"
        print(_name)
        train_save_classification(nodes_hidden=hidden_size, learning_rate=lr, name=_name, dropout=True)


2023-02-28 16:39:20,881 [WARNING] Scheduler Object not found amongst the Arguments. Ignore warning if scheduler wasn't meant to be in the loop


drop_800_1e-03


2023-02-28 16:39:22,591 [INFO] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-02-28 16:39:22,592 [INFO] NumExpr defaulting to 8 threads.


The tensorboard model name corresponding to this model is drop_800_1e-03_20230228163920881925
Epoch 1/600
----------


100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:06<00:00, 60.20it/s]


...

train Loss: 0.0413
train Acc: 0.9860


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 66.55it/s]


val Loss: 0.0547
val Acc: 0.9851


100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:01<00:00, 74.11it/s]

test Loss: 0.0500
test Acc: 0.9840

Training completed in 1h 18m 55s


In [ ]:
# Loading the model again and continuing training is possible
"""
new_model = MLPModel(input_dim=784, output_dim=10, hidden_layer=400)
new_model.load_state_dict(torch.load('weights/400_1e-03.pth'))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(new_model.parameters(),lr=1e-3)

args = {
        'model': new_model,
        'train_dataset': train_mnist,
        'criterion': criterion,
        'batch_size': 128,
        'optimizer': optimizer,
        'es_flag': False,
        'num_epochs': 10,
        'val_dataset': val_mnist,
        'mode': 2
        }
trainer = TrainModelWrapper(**args)
model, history = trainer.train()
"""